In [1]:
import skimage as ski
import numpy as np
import openpyxl as xl
import csv
import os
import xmltodict

In [2]:

"""
for this typical ETL pattern, this notebook explores EXTRACT

"""

'\nfor this typical ETL pattern, this notebook explores EXTRACT\n\n'

In [4]:
plate1_path='/Volumes/GoogleDrive/My Drive/ELISAarrayReader/images_scienion/2020-01-15_plate4_AEP_Feb3_6mousesera'
# plate1_path = 'Plates_given_to_manu/2020-01-15_plate4_AEP_Feb3_6mousesera'
plate1_gal = plate1_path+os.sep+'8x6_test.gal'
plate1_xml = plate1_path+os.sep+'8x6_test.conf.xml'


In [3]:
"""
explore the .gal file
"""
# explore the data format, delimiter type
with open(plate1_gal, newline='') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter='\t')
    for row in csv_reader:
        print(row)

['ATF', '1.0']
['5', '5']
['Type=GenePix ArrayList V1.0']
['BlockCount=1']
['BlockType=0']
['Supplier=Scienion AG']
['Block1= 1100, 1100, 200, 8, 405, 6, 410']
['Block', 'Row', 'Column', 'ID', 'Name']
['1', '1', '1', 'Marker', 'Marker']
['1', '2', '1', 'Marker', 'Marker']
['1', '3', '1', 'spot', 'spot']
['1', '4', '1', 'spot', 'spot']
['1', '5', '1', 'spot', 'spot']
['1', '6', '1', 'Marker', 'Marker']
['1', '1', '2', 'spot', 'spot']
['1', '2', '2', 'spot', 'spot']
['1', '3', '2', 'spot', 'spot']
['1', '4', '2', 'spot', 'spot']
['1', '5', '2', 'spot', 'spot']
['1', '6', '2', 'spot', 'spot']
['1', '1', '3', 'spot', 'spot']
['1', '2', '3', 'spot', 'spot']
['1', '3', '3', 'spot', 'spot']
['1', '4', '3', 'spot', 'spot']
['1', '5', '3', 'spot', 'spot']
['1', '6', '3', 'spot', 'spot']
['1', '1', '4', 'spot', 'spot']
['1', '2', '4', 'spot', 'spot']
['1', '3', '4', 'spot', 'spot']
['1', '4', '4', 'spot', 'spot']
['1', '5', '4', 'spot', 'spot']
['1', '6', '4', 'spot', 'spot']
['1', '1', '5', 'sp

In [4]:

# block starts as list, then will be cast to numpy array
with open(plate1_gal, newline='') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter='\t')
    out_dict = {}
    out_dict['Header'] = []
    out_dict['Block'] = []
    begin_block = False
    for row in csv_reader:
        if row[0] == 'Block':
            begin_block = True
        
        # still reading header values
        if not begin_block:
            out_dict['Header'].append(row)
            
        # begin block (spot) mapping
        if begin_block:
            out_dict['Block'].append(row)

In [5]:
out_dict['Block']


[['Block', 'Row', 'Column', 'ID', 'Name'],
 ['1', '1', '1', 'Marker', 'Marker'],
 ['1', '2', '1', 'Marker', 'Marker'],
 ['1', '3', '1', 'spot', 'spot'],
 ['1', '4', '1', 'spot', 'spot'],
 ['1', '5', '1', 'spot', 'spot'],
 ['1', '6', '1', 'Marker', 'Marker'],
 ['1', '1', '2', 'spot', 'spot'],
 ['1', '2', '2', 'spot', 'spot'],
 ['1', '3', '2', 'spot', 'spot'],
 ['1', '4', '2', 'spot', 'spot'],
 ['1', '5', '2', 'spot', 'spot'],
 ['1', '6', '2', 'spot', 'spot'],
 ['1', '1', '3', 'spot', 'spot'],
 ['1', '2', '3', 'spot', 'spot'],
 ['1', '3', '3', 'spot', 'spot'],
 ['1', '4', '3', 'spot', 'spot'],
 ['1', '5', '3', 'spot', 'spot'],
 ['1', '6', '3', 'spot', 'spot'],
 ['1', '1', '4', 'spot', 'spot'],
 ['1', '2', '4', 'spot', 'spot'],
 ['1', '3', '4', 'spot', 'spot'],
 ['1', '4', '4', 'spot', 'spot'],
 ['1', '5', '4', 'spot', 'spot'],
 ['1', '6', '4', 'spot', 'spot'],
 ['1', '1', '5', 'spot', 'spot'],
 ['1', '2', '5', 'spot', 'spot'],
 ['1', '3', '5', 'spot', 'spot'],
 ['1', '4', '5', 'spot', 's

In [6]:
# parse the index for various header values
# ['Block', 'Row', 'Column', 'ID', 'Name']
row_idx = [int(idx) for idx, value in enumerate(out_dict['Block'][0]) if value == 'Row'][0]
col_idx = [int(idx) for idx, value in enumerate(out_dict['Block'][0]) if value == 'Column'][0]
id_idx = [int(idx) for idx, value in enumerate(out_dict['Block'][0]) if value == 'ID'][0]
name_idx = [int(idx) for idx, value in enumerate(out_dict['Block'][0]) if value == 'Name'][0]


In [7]:
# find the max rows and cols
max_row = 0
max_col = 0
for spot in out_dict['Block'][1:]:
    if max_row < int(spot[row_idx]):
        max_row = int(spot[row_idx])
    if max_col < int(spot[col_idx]):
        max_col = int(spot[col_idx])

gal_spot_array_list_ID = [[[None] for i in range(max_col)] for j in range(max_row)]
gal_spot_array_numpy_ID = np.empty(shape=(max_row, max_col), dtype=np.dtype('U100'))

gal_spot_array_list_Name = [[[None] for i in range(max_col)] for j in range(max_row)]
gal_spot_array_numpy_Name = np.empty(shape=(max_row, max_col), dtype=np.dtype('U100'))


In [8]:

for spot in out_dict['Block'][1:]:
    r = int(spot[row_idx])-1
    c = int(spot[col_idx])-1
    ID = spot[id_idx]
    name = spot[name_idx]
    
    gal_spot_array_list_ID[r][c] = ID
    gal_spot_array_numpy_ID[r, c] = ID
    
    gal_spot_array_list_Name[r][c] = name
    gal_spot_array_numpy_Name[r, c] = name


In [5]:
"""
explore the .xml file
"""

# xmltodict is much easier to use and translate than xml.dom.minidom
with open(plate1_xml) as fd:
    doc = xmltodict.parse(fd.read())

# layout of array
layout = doc['configuration']['well_configurations']['configuration']['array']['layout']
# fiducials
fiduc = layout['marker']

# spot IDs
spots = doc['configuration']['well_configurations']['configuration']['array']['spots']['spot']

# replicates
repl = doc['configuration']['well_configurations']['configuration']['array']['spots']['multiplet']


In [6]:
rows = int(layout['@rows'])
columns = int(layout['@cols'])
v_pitch = float(layout['@vspace'])
h_pitch = float(layout['@hspace'])
spot_width = float(layout['@expected_diameter'])
bg_offset = float(layout['@background_offset'])
bg_thickness = float(layout['@background_thickness'])
max_diam = float(layout['@max_diameter'])
min_diam = float(layout['@min_diameter'])

In [7]:

# xml_spot_array_list = [[[None] for i in range(columns)] for j in range(rows)]
xml_spot_array_numpy = np.empty(shape=(rows, columns), dtype=np.dtype('U100'))

# xml_spot_array_list_ID = [[[None] for i in range(columns)] for j in range(rows)]
xml_spot_array_numpy_ID = np.empty(shape=(rows, columns), dtype=np.dtype('U100'))

# xml_spot_array_list_antigen = [[[None] for i in range(columns)] for j in range(rows)]
xml_spot_array_numpy_antigen = np.empty(shape=(rows, columns), dtype=np.dtype('U100'))

In [8]:

for spot in spots:
    r = int(spot['@row'])
    c = int(spot['@col'])
    v = spot['@spot_type']
    ID = spot['@id']
    
    xml_spot_array_list[r][c] = v
    xml_spot_array_numpy[r,c] = v
    
    xml_spot_array_list_ID[r][c] = ID
    xml_spot_array_numpy_ID[r,c] = ID

for f in fiduc:
    r = int(f['@row'])
    c = int(f['@col'])
    v = f['@spot_type']
    # fiduc do not have "ID"

    xml_spot_array_list[r][c] = v
    xml_spot_array_numpy[r,c] = v


In [13]:
xml_spot_array_numpy

array([['Reference, Diagnostic', 'Diagnostic', 'Diagnostic',
        'Diagnostic', 'Diagnostic', 'Diagnostic', 'Diagnostic',
        'Reference, Diagnostic'],
       ['Reference, Diagnostic', 'Diagnostic', 'Diagnostic',
        'Diagnostic', 'Diagnostic', 'Diagnostic', 'Diagnostic',
        'Reference, PositiveControl'],
       ['Diagnostic', 'Diagnostic', 'Diagnostic', 'Diagnostic',
        'Diagnostic', 'Diagnostic', 'Diagnostic', 'Diagnostic'],
       ['Diagnostic', 'Diagnostic', 'Diagnostic', 'Diagnostic',
        'Diagnostic', 'Diagnostic', 'Diagnostic', 'Diagnostic'],
       ['Diagnostic', 'Diagnostic', 'Diagnostic', 'Diagnostic',
        'Diagnostic', 'Diagnostic', 'Diagnostic',
        'Reference, NegativeControl'],
       ['Reference, Diagnostic', 'Diagnostic', 'Diagnostic',
        'Diagnostic', 'Diagnostic', 'Diagnostic', 'Diagnostic',
        'Reference, Diagnostic']], dtype='<U100')

In [9]:

# walk through the replicates and assign the repl @id to the array
# 1) iterate repl
# 2) extract all_ids (not @id), extract antigen (@id)
# 3) find cells that correspond to each in all_ids
# 4) use the xml_spot_array_list_ID to assign by antigen to ID
ids = xml_spot_array_numpy_ID
anti = xml_spot_array_numpy_antigen

for rep in repl:
    antigen = rep['@id']
    all_spots = rep['id'] # list of IDs
    for spot in all_spots:
        anti[np.where(ids==spot)] = antigen
        
xml_spot_array_numpy_antigen = anti

In [15]:
xml_spot_array_numpy_antigen

array([['', '114', '100', 'KZ52', 'c13C6', 'c2G4', 'c4G7', ''],
       ['', '114', '100', 'KZ52', 'c13C6', 'c2G4', 'c4G7', ''],
       ['anti-HIS tag', '15731', '15742', '15750', '15878', '15946',
        '15960', 'Q411'],
       ['anti-HIS tag', '15731', '15742', '15750', '15878', '15946',
        '15960', 'Q411'],
       ['anti-HIS tag', '15974', '16061', 'FVM04', 'VIC122', 'Q206',
        'Q314', ''],
       ['', '15974', '16061', 'FVM04', 'VIC122', 'Q206', 'Q314', '']],
      dtype='<U100')

In [28]:
"""
The above code should parse the two files (.gal, .xml) into two data formats:
.gal
    gal_spot_array_list_ID[r][c] = ID
    gal_spot_array_numpy_ID[r, c] = ID
    
    gal_spot_array_list_Name[r][c] = name
    gal_spot_array_numpy_Name[r, c] = name
    
.xml
    xml_spot_array_list[r][c] = spot type
    xml_spot_array_numpy[r,c] = spot type
    
    xml_spot_array_list_ID[r][c] = ID
    xml_spot_array_numpy_ID[r,c] = ID
    
    xml_spot_array_numpy_antigen = antigen
    
the 'numpy' and 'list' forms are identical
each can be sliced and are indexed by row-column format
each has a value that is a simple string

So we can use the following three arrays to query type, cell ID name, and antigen name
    xml_spot_array_numpy
    xml_spot_array_numpy_ID
    xml_spot_array_numpy_antigen

"""

"\nThe above code should parse the two files (.gal, .xml) into two data formats:\n.gal\n    gal_spot_array_list_ID[r][c] = ID\n    gal_spot_array_numpy_ID[r, c] = ID\n    \n    gal_spot_array_list_Name[r][c] = name\n    gal_spot_array_numpy_Name[r, c] = name\n    \n.xml\n    xml_spot_array_list[r][c] = spot type\n    xml_spot_array_numpy[r,c] = spot type\n    \n    xml_spot_array_list_ID[r][c] = ID\n    xml_spot_array_numpy_ID[r,c] = ID\n    \n    xml_spot_array_numpy_antigen = antigen\n    \nthe 'numpy' and 'list' forms are identical\neach can be sliced and are indexed by row-column format\neach has a value that is a simple string\n\nSo we can use the following three arrays to query control, ID name, and antigen name\n\n"

In [11]:
ids

array([['', 'spot-1-2', 'spot-1-3', 'spot-1-4', 'spot-1-5', 'spot-1-6',
        'spot-1-7', ''],
       ['', 'spot-2-2', 'spot-2-3', 'spot-2-4', 'spot-2-5', 'spot-2-6',
        'spot-2-7', 'spot-2-8'],
       ['spot-3-1', 'spot-3-2', 'spot-3-3', 'spot-3-4', 'spot-3-5',
        'spot-3-6', 'spot-3-7', 'spot-3-8'],
       ['spot-4-1', 'spot-4-2', 'spot-4-3', 'spot-4-4', 'spot-4-5',
        'spot-4-6', 'spot-4-7', 'spot-4-8'],
       ['spot-5-1', 'spot-5-2', 'spot-5-3', 'spot-5-4', 'spot-5-5',
        'spot-5-6', 'spot-5-7', 'spot-5-8'],
       ['', 'spot-6-2', 'spot-6-3', 'spot-6-4', 'spot-6-5', 'spot-6-6',
        'spot-6-7', '']], dtype='<U100')

In [2]:
xml_spot_array_numpy_antigen.shape


NameError: name 'xml_spot_array_numpy_antigen' is not defined